kaggle 데이터를 불러오기

In [ ]:
! pip install kaggle
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

from google.colab import files
files.upload()

Uninstalling kaggle-1.5.10:
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 1.5MB 13.2MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72858 sha256=ab068131fddd991600c210bd779a53aa7bebc6e9f9d8916966f5795477711c22
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"osmosm7","key":"3fd0923f2fa62dda0b5b3f50726b3935"}'}

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
# kaggle.json 파일 이동
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 일어나지 않도록
!chmod 600~/.kaggle/kaggle.json

# 본인이 참가한 모든 대회 보기
! kaggle competitions list

chmod: missing operand after ‘600~/.kaggle/kaggle.json’
Try 'chmod --help' for more information.
ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        105           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        214           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        324           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2768           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      21408            True  
house-prices-advanced-regression-te

In [ ]:
# API를 이용해 데이터 다운

!kaggle competitions download -c humpback-whale-identification

100% 5.50G/5.51G [01:57<00:00, 41.6MB/s]
100% 5.51G/5.51G [01:57<00:00, 50.5MB/s]


In [ ]:
!ls

humpback-whale-identification.zip  kaggle.json	sample_data


In [ ]:
# 데이터 압축 해제
!unzip humpback-whale-identification.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/cd2c4ae6b.jpg     
  inflating: train/cd310f07c.jpg     
  inflating: train/cd32697ae.jpg     
  inflating: train/cd346388a.jpg     
  inflating: train/cd36f1901.jpg     
  inflating: train/cd382b662.jpg     
  inflating: train/cd3937f27.jpg     
  inflating: train/cd3a073a2.jpg     
  inflating: train/cd3afe416.jpg     
  inflating: train/cd407fae0.jpg     
  inflating: train/cd437a15c.jpg     
  inflating: train/cd444321e.jpg     
  inflating: train/cd4740ee9.jpg     
  inflating: train/cd49e7801.jpg     
  inflating: train/cd4b8f33f.jpg     
  inflating: train/cd4fae084.jpg     
  inflating: train/cd520d861.jpg     
  inflating: train/cd52c565c.jpg     
  inflating: train/cd5577b32.jpg     
  inflating: train/cd5712905.jpg     
  inflating: train/cd598b557.jpg     
  inflating: train/cd59d5874.jpg     
  inflating: train/cd5a9c5b1.jpg     
  inflating: train/cd5b23ddc.jpg     
  inflating: train/cd5dd712e.jpg     
  inflating: t

# 필요 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms

from sklearn.preprocessing import LabelEncoder, OneHotEncoder


import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import joblib

import os


from PIL import Image

 ## GPU 설정

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
device

device(type='cuda')

## 데이터 확인

In [ ]:
train_label = pd.read_csv('train.csv')
NCLASSES = len(train_label['Id'].unique())

train_label.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


### 라벨 생성

In [ ]:
def prepare_labels(y):
    # From here: https://www.kaggle.com/pestipeti/keras-cnn-starter
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    y = onehot_encoded
    return y, label_encoder

In [ ]:
y, label_encoder = prepare_labels(train_label['Id'])

In [ ]:
train_label['fold'] = 1

In [ ]:
# cross validation을 위해 Stratified K fold 를 해준다
from sklearn.model_selection import StratifiedKFold as skfold
skf = skfold(n_splits = 6, random_state = 42, shuffle = False)

for i, (train_index, vld_index) in enumerate(skf.split(train_label['Image'],train_label['Id'])):
     train_label.loc[vld_index,'fold'] = i

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)


In [ ]:
train_label

,Image,Id,fold
0,0000e88ab.jpg,w_f48451c,0
1,0001f9222.jpg,w_c3d896a,2
2,00029d126.jpg,w_20df2c5,0
3,00050a15a.jpg,new_whale,0
4,0005c1ef8.jpg,new_whale,0
...,...,...,...
25356,ffef89eed.jpg,w_9c506f6,5
25357,fff7faf61.jpg,w_9cf0388,5
25358,fff9002e0.jpg,w_bd1c3d5,5
25359,fffcde6fe.jpg,w_9f30885,4


## 전처리 설정

In [ ]:
# cross validation을 위해 Stratified K fold 를 해준다
from sklearn.model_selection import StratifiedKFold as skfold
skf = skfold(n_splits = 6, random_state = 42, shuffle = False)

for i, (train_index, vld_index) in enumerate(skf.split(train_label['Image'],train_label['Id'])):
     train_label.loc[vld_index,'fold'] = i
trn_fold = [i for i in range(5)]
vld_fold = [5]
trn_idx = train_label.loc[train_label['fold'].isin(trn_fold)].index
vld_idx = train_label.loc[train_label['fold'].isin(vld_fold)].index

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)


In [ ]:
train_label.loc[trn_idx].head()

,Image,Id,fold
0,0000e88ab.jpg,w_f48451c,0
1,0001f9222.jpg,w_c3d896a,2
2,00029d126.jpg,w_20df2c5,0
3,00050a15a.jpg,new_whale,0
4,0005c1ef8.jpg,new_whale,0


In [ ]:
train_label.loc[vld_idx].head()

,Image,Id,fold
243,026345181.jpg,w_2e4af64,5
259,028f11c08.jpg,w_55e0064,5
324,03325fea5.jpg,w_65dadc8,5
352,037ce9282.jpg,w_f1d2902,5
398,03efb0fdb.jpg,w_178144f,5


In [ ]:
test_df = [img for img in os.listdir('test/')]

### 데이터 로더를 위한 데이터 셋 설정

In [ ]:
class WhaleDataset():
    def __init__(self,image_folder, process = 'train',df = None, transform = None, y = None):
        self.image_folder = image_folder
        self.df = df.reset_index()
        self.imgs_list = df['Image'].values
        self.process = process
        self.transform = transform
        self.y = y
        if self.process =='train':
            self.df = df.values

    def __len__(self):
        return len(self.imgs_list)

    def __getitem__(self,idx):
        if self.process =='train':
            img_name = os.path.join(self.image_folder,self.imgs_list[idx])
            label = self.y[idx]

        elif self.process =='test':
            img_name = os.path.join(self.image_folder, self.imgs_list[idx])
            label = np.zeros((NCLASSES,))

        img = Image.open(img_name).convert('RGB')
        img = self.transform(img)
        if self.process == 'train':
            return img,label
        elif self.process == 'test':
            return img,label, self.imgs_list[idx]

### 데이터 어그멘테이션, 데이터 로더 설졍 

In [ ]:
train_augmentation = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.485,0.485],[0.229,0.224,0.225])
])


# Validation set은 별다른 augmetation 노노
valid_augmentation = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.485,0.485],[0.229,0.224,0.225])
])

test_augmentation = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.485,0.485],[0.229,0.224,0.225])
])

#def __init__(self,image_folder, porcess = 'train',df = None, transform = None, y = None):

train_data = WhaleDataset('train/', process = 'train',df = train_label.loc[trn_idx], transform = train_augmentation, y = y[trn_idx])

valid_data = WhaleDataset('train/', process = 'train', df = train_label.loc[vld_idx], transform = valid_augmentation, y =y[vld_idx])

test_data = WhaleDataset('test/', process = 'test', df = pd.DataFrame(test_df,columns= ['Image']), transform = test_augmentation)

In [ ]:
trn_loader = DataLoader(train_data,
                       shuffle = True,
                       num_workers = 2,
                       batch_size = 128)

vld_loader = DataLoader(valid_data,
                        shuffle = False,
                       num_workers = 2,
                       batch_size = 128)

test_loader = DataLoader(test_data,
                        shuffle = False,
                       num_workers = 2,
                       batch_size = 128)

### pre_trained 모델 가져오기

In [ ]:
model_ft = models.resnet18(pretrained = True)

In [ ]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Pretrained model output 변경해주기

In [ ]:
model_ft = models.resnet18(pretrained = True)

# pretrained model 의 in_features
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs,len(set(train_label['Id'])))

model_ft = model_ft.to(device)


손실함수, 옵티마이저, 스케쥴러

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

#optimizer_ft = optim.SGD(model_ft.parameters(), lr = 0.001,momentum = 0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr = 0.001)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft,
                                            mode = 'max',
                                           verbose = True,
                                           patience = 7,
                                           factor = 0.5)

In [ ]:
from tqdm import tqdm

### 학습

In [ ]:
best_score = 0
for e in range(10):
    total = 0
    correct = 0
    train_loss = []
    model_ft.train()

    for i, (inputs, targets) in tqdm(enumerate(trn_loader),total = len(trn_loader)):
        #gpu사용시
        inputs = inputs.to(device) 
        targets = targets.to(device)
        
        logits = model_ft(inputs)

        loss = loss_fn(logits,targets.float())


        loss.backward(retain_graph=True)

        optimizer_ft.step()
        optimizer_ft.zero_grad()
        train_loss.append(loss.item())




    val_loss = []
    val_true = []
    val_pred = []

    model_ft.eval()


    # gradient를 건들지 않겠다.
    with torch.no_grad():
        for inputs, targets in vld_loader:
        # gpu사용시
            inputs = inputs.cuda() 
            targets = targets.cuda()
            logits = model_ft(inputs)
            loss = loss_fn(logits,targets.float())

            val_loss.append(loss.item())

            pred =logits.to('cpu').argmax(dim=1).data.numpy()
            target = targets.to('cpu').argmax(dim=1).data.numpy()
            
            total +=len(pred)
            correct += (pred == target).sum().item()
    print(f'Epoch {e}, train loss: {np.mean(train_loss):.4f}')
    print(f'Epoch {e}, valid loss: {np.mean(val_loss):.4f}')


    score = (100 * correct / total )
    #score = recall_score(val_trues,val_preds,average = 'micro')
    print(score)
    
    if score> best_score:
        best_score = score

        state_dict = model_ft.state_dict()
        model = model_ft
        torch.save(state_dict,'model.pt')



  0%|          | 0/166 [00:00<?, ?it/s]

  1%|          | 1/166 [00:03<08:22,  3.05s/it]

  1%|          | 2/166 [00:03<06:09,  2.25s/it]

  2%|▏         | 3/166 [00:06<06:22,  2.35s/it]

  2%|▏         | 4/166 [00:06<04:46,  1.77s/it]

  3%|▎         | 5/166 [00:08<05:19,  1.98s/it]

  4%|▎         | 6/166 [00:09<04:00,  1.50s/it]

  4%|▍         | 7/166 [00:11<04:46,  1.80s/it]

  5%|▍         | 8/166 [00:12<03:40,  1.40s/it]

  5%|▌         | 9/166 [00:14<04:37,  1.77s/it]

  6%|▌         | 10/166 [00:15<03:32,  1.36s/it]

  7%|▋         | 11/166 [00:17<04:18,  1.67s/it]

  7%|▋         | 12/166 [00:18<03:23,  1.32s/it]

  8%|▊         | 13/166 [00:20<04:20,  1.70s/it]

  8%|▊         | 14/166 [00:21<03:20,  1.32s/it]

  9%|▉         | 15/166 [00:23<04:21,  1.73s/it]

 10%|▉         | 16/166 [00:24<03:23,  1.36s/it]

 10%|█         | 17/166 [00:26<04:08,  1.67s/it]

 11%|█         | 18/166 [00:27<03:20,  1.35s/it]

 11%|█▏        | 19/166 [00:29<04:02,  1.65s/it]

 12%|█▏        |

Epoch 0, train loss: 0.0013
Epoch 0, valid loss: 0.0013
38.121154756270705




  0%|          | 0/166 [00:00<?, ?it/s]

  1%|          | 1/166 [00:03<09:02,  3.29s/it]

  1%|          | 2/166 [00:03<06:37,  2.42s/it]

  2%|▏         | 3/166 [00:06<06:35,  2.43s/it]

  2%|▏         | 4/166 [00:06<04:53,  1.81s/it]

  3%|▎         | 5/166 [00:09<05:32,  2.07s/it]

  4%|▎         | 6/166 [00:09<04:10,  1.57s/it]

  4%|▍         | 7/166 [00:12<05:04,  1.91s/it]

  5%|▍         | 8/166 [00:12<03:50,  1.46s/it]

  5%|▌         | 9/166 [00:15<04:37,  1.77s/it]

  6%|▌         | 10/166 [00:15<03:32,  1.36s/it]

  7%|▋         | 11/166 [00:18<04:38,  1.80s/it]

  7%|▋         | 12/166 [00:18<03:31,  1.37s/it]

  8%|▊         | 13/166 [00:21<04:35,  1.80s/it]

  8%|▊         | 14/166 [00:21<03:29,  1.38s/it]

  9%|▉         | 15/166 [00:24<04:26,  1.76s/it]

 10%|▉         | 16/166 [00:25<03:22,  1.35s/it]

 10%|█         | 17/166 [00:27<04:18,  1.73s/it]

 11%|█         | 18/166 [00:28<03:18,  1.34s/it]

 11%|█▏        | 19/166 [00:30<04:13,  1.73s/it]

 12%|█▏        |

Epoch 1, train loss: 0.0013
Epoch 1, valid loss: 0.0013
38.121154756270705




  0%|          | 0/166 [00:00<?, ?it/s]

  1%|          | 1/166 [00:03<09:30,  3.45s/it]

  1%|          | 2/166 [00:03<06:55,  2.53s/it]

  2%|▏         | 3/166 [00:06<06:59,  2.57s/it]

  2%|▏         | 4/166 [00:06<05:12,  1.93s/it]

  3%|▎         | 5/166 [00:09<05:42,  2.12s/it]

  4%|▎         | 6/166 [00:09<04:16,  1.60s/it]

  4%|▍         | 7/166 [00:12<05:05,  1.92s/it]

  5%|▍         | 8/166 [00:12<03:52,  1.47s/it]

  5%|▌         | 9/166 [00:15<04:50,  1.85s/it]

  6%|▌         | 10/166 [00:16<03:40,  1.41s/it]

  7%|▋         | 11/166 [00:18<04:45,  1.84s/it]

  7%|▋         | 12/166 [00:19<03:36,  1.41s/it]

  8%|▊         | 13/166 [00:21<04:27,  1.75s/it]

  8%|▊         | 14/166 [00:22<03:24,  1.35s/it]

  9%|▉         | 15/166 [00:25<04:29,  1.79s/it]

 10%|▉         | 16/166 [00:25<03:25,  1.37s/it]

 10%|█         | 17/166 [00:28<04:33,  1.83s/it]

 11%|█         | 18/166 [00:28<03:28,  1.41s/it]

 11%|█▏        | 19/166 [00:31<04:26,  1.82s/it]

 12%|█▏        |

...

Epoch 7, train loss: 0.0011
Epoch 7, valid loss: 0.0012
38.26313298627544




  0%|          | 0/166 [00:00<?, ?it/s]

  1%|          | 1/166 [00:03<09:12,  3.35s/it]

  1%|          | 2/166 [00:03<06:43,  2.46s/it]

  2%|▏         | 3/166 [00:06<06:57,  2.56s/it]

  2%|▏         | 4/166 [00:06<05:10,  1.92s/it]

  3%|▎         | 5/166 [00:09<05:47,  2.16s/it]

  4%|▎         | 6/166 [00:10<04:21,  1.64s/it]

  4%|▍         | 7/166 [00:12<05:18,  2.00s/it]

  5%|▍         | 8/166 [00:13<04:00,  1.52s/it]

  5%|▌         | 9/166 [00:15<04:50,  1.85s/it]

  6%|▌         | 10/166 [00:16<03:41,  1.42s/it]

  7%|▋         | 11/166 [00:18<04:31,  1.75s/it]

  7%|▋         | 12/166 [00:19<03:29,  1.36s/it]

  8%|▊         | 13/166 [00:22<04:44,  1.86s/it]

  8%|▊         | 14/166 [00:22<03:35,  1.42s/it]

  9%|▉         | 15/166 [00:25<04:28,  1.78s/it]

 10%|▉         | 16/166 [00:25<03:25,  1.37s/it]

 10%|█         | 17/166 [00:28<04:28,  1.80s/it]

 11%|█         | 18/166 [00:29<03:24,  1.38s/it]

 11%|█▏        | 19/166 [00:31<04:25,  1.80s/it]

 12%|█▏        |

Epoch 8, train loss: 0.0011
Epoch 8, valid loss: 0.0012
34.78466635115949




  0%|          | 0/166 [00:00<?, ?it/s]

  1%|          | 1/166 [00:03<09:22,  3.41s/it]

  1%|          | 2/166 [00:03<06:50,  2.50s/it]

  2%|▏         | 3/166 [00:06<06:55,  2.55s/it]

  2%|▏         | 4/166 [00:06<05:08,  1.90s/it]

  3%|▎         | 5/166 [00:09<05:49,  2.17s/it]

  4%|▎         | 6/166 [00:10<04:21,  1.63s/it]

  4%|▍         | 7/166 [00:12<05:10,  1.96s/it]

  5%|▍         | 8/166 [00:13<03:55,  1.49s/it]

  5%|▌         | 9/166 [00:15<04:56,  1.89s/it]

  6%|▌         | 10/166 [00:16<03:43,  1.44s/it]

  7%|▋         | 11/166 [00:19<04:52,  1.89s/it]

  7%|▋         | 12/166 [00:19<03:41,  1.44s/it]

  8%|▊         | 13/166 [00:22<04:43,  1.85s/it]

  8%|▊         | 14/166 [00:22<03:35,  1.42s/it]

  9%|▉         | 15/166 [00:25<04:30,  1.79s/it]

 10%|▉         | 16/166 [00:25<03:25,  1.37s/it]

 10%|█         | 17/166 [00:28<04:29,  1.81s/it]

 11%|█         | 18/166 [00:29<03:25,  1.39s/it]

 11%|█▏        | 19/166 [00:31<04:11,  1.71s/it]

 12%|█▏        |

Epoch 9, train loss: 0.0010
Epoch 9, valid loss: 0.0012
35.896829152863226


In [ ]:
print(f'Epoch {e}, train loss: {np.mean(train_loss):.4f}')
print(f'Epoch {e}, valid loss: {np.mean(val_loss):.4f}')

Epoch 0, train loss: 0.0155
Epoch 0, valid loss: 0.0013


### Submission

In [ ]:
sub = pd.read_csv('sample_submission.csv')

model_ft.eval()
for (data, target, name) in tqdm(test_loader):
    data = data.cuda()
    output = model_ft(data)
    output = output.cpu().detach().numpy()
    for i, (e, n) in enumerate(list(zip(output, name))):
        sub.loc[sub['Image'] == n, 'Id'] = ' '.join(label_encoder.inverse_transform(e.argsort()[-5:][::-1]))
        
sub.to_csv('submission.csv', index=False)



  0%|          | 0/63 [00:00<?, ?it/s]

  2%|▏         | 1/63 [00:03<04:00,  3.88s/it]

  3%|▎         | 2/63 [00:04<02:54,  2.86s/it]

  5%|▍         | 3/63 [00:07<02:51,  2.86s/it]

  6%|▋         | 4/63 [00:07<02:06,  2.15s/it]

  8%|▊         | 5/63 [00:10<02:17,  2.38s/it]

 10%|▉         | 6/63 [00:11<01:43,  1.81s/it]

 11%|█         | 7/63 [00:14<02:02,  2.19s/it]

 13%|█▎        | 8/63 [00:14<01:33,  1.70s/it]

 14%|█▍        | 9/63 [00:17<01:48,  2.01s/it]

 16%|█▌        | 10/63 [00:18<01:23,  1.57s/it]

 17%|█▋        | 11/63 [00:20<01:43,  1.98s/it]

 19%|█▉        | 12/63 [00:21<01:19,  1.55s/it]

 21%|██        | 13/63 [00:24<01:37,  1.96s/it]

 22%|██▏       | 14/63 [00:24<01:14,  1.52s/it]

 24%|██▍       | 15/63 [00:27<01:32,  1.93s/it]

 25%|██▌       | 16/63 [00:28<01:10,  1.50s/it]

 27%|██▋       | 17/63 [00:30<01:24,  1.83s/it]

 29%|██▊       | 18/63 [00:31<01:04,  1.44s/it]

 30%|███       | 19/63 [00:34<01:22,  1.88s/it]

 32%|███▏      | 20/63 [00:34<01:03,